In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import scipy as sp
import warnings
warnings.filterwarnings('ignore')

1. Have the size of the sets changed over time?

In [176]:
df_sets = pd.read_csv("/kaggle/input/lego-database/sets.csv")
df_sets.head()

,set_num,name,year,theme_id,num_parts
0,00-1,Weetabix Castle,1970,414,471
1,0011-2,Town Mini-Figures,1978,84,12
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,2
3,0012-1,Space Mini-Figures,1979,143,12
4,0013-1,Space Mini-Figures,1979,143,12


In [177]:
df_sets.sort_values(by='num_parts', ascending = False)[0:4]

,set_num,name,year,theme_id,num_parts
170,10189-1,Taj Mahal,2008,276,5922
11614,SWMP-1,Star Wars / M&M Mosaic - Promo Set,2005,169,5461
1337,2000409-1,Window Exploration Bag,2010,507,5200
161,10179-1,Millennium Falcon - UCS,2007,174,5195


In [217]:
df_sets_per_year = df_sets.groupby('year').agg(mean_part=('num_parts', np.mean),
                                        std=('num_parts', np.std),
                                        theme_nbr = ('theme_id' , pd.Series.nunique),
                                        set_nbr = ('set_num', pd.Series.nunique))

Fig_1 = px.scatter(df_sets_per_year,
                   x = df_sets_per_year.index,
                   y = 'mean_part',
                   labels = {'year':'Year',
                          'mean_part' : "Average number of part",
                          'theme_nbr' : 'Number of theme'},
                   title = "Evolution of the mean size of LEGO's set from 1950",
                   height = 600,
                   width = 1200)

Fig_1.update_traces(marker=dict(size=10)
                   )

Fig_1.show()

if p < 0.05:
    print("The evolution of the size of LEGO's set over time is significant.")
else:
    print("There has not been significant changes in the size of LEGO's set over time.")

The evolution of the size of LEGO's set over time is significant.


In [194]:
Fig_1 = px.scatter(df_sets_per_year,
                   x = df_sets_per_year.index,
                   y = 'set_nbr',
                   color = 'theme_nbr',
                   labels = {'year':'Year',
                          'theme_nbr' : "Number of themes",
                          'set_nbr':'Number of sets'},
                   title = "Number of sets per year",
                   color_continuous_scale=px.colors.sequential.Viridis,
                   height = 600,
                   width = 1200,
                   size = 'theme_nbr')


Fig_1.show()

In [195]:
Fig_1 = px.scatter(df_sets_per_year,
                   x = 'theme_nbr',
                   y = 'set_nbr',
                   color = 'mean_part',
                   size = 'mean_part',
                   labels = {'set_nbr':'Number of sets',
                          'mean_part' : "Average number of parts",
                          'theme_nbr' : 'Number of themes'},
                   height = 600,
                   width = 800)


Fig_1.show()

In [196]:
df_themes = pd.read_csv("/kaggle/input/lego-database/themes.csv")
df_themes.head()

,id,name,parent_id
0,1,Technic,NaN
1,2,Arctic Technic,1.0
2,3,Competition,1.0
3,4,Expert Builder,1.0
4,5,Model,1.0


In [197]:
df_merged = pd.merge(df_sets, df_themes, left_on = 'theme_id', right_on = 'id')
df_merged.drop(['theme_id', 'id', 'parent_id'], axis = 1, inplace = True)
df_merged.rename(columns = {'name_x' : 'set',
                            'name_y' : 'theme'}, inplace = True)
df_merged.head()

,set_num,set,year,num_parts,theme
0,00-1,Weetabix Castle,1970,471,Castle
1,0011-2,Town Mini-Figures,1978,12,Supplemental
2,0011-3,Castle 2 for 1 Bonus Offer,1987,2,Lion Knights
3,0012-1,Space Mini-Figures,1979,12,Supplemental
4,0013-1,Space Mini-Figures,1979,12,Supplemental


In [224]:
sets_size_theme = df_merged.groupby(['theme']).agg(mean_part = ('num_parts' , 'mean'), std = ('num_parts' , 'std'), set_nbr = ('set', pd.Series.nunique)).sort_values(by = 'mean_part', ascending = False)[:15]

Fig = px.scatter(sets_size_theme,
                 y = sets_size_theme.index,
                 x = sets_size_theme['mean_part'],
                 error_x = 'std',
                 color = sets_size_theme.index,
                 text = 'set_nbr',
                 height = 500,
                 width = 800,
                 title = 'TOP 15 of largest themes',
                 labels = {'theme':'Theme',
                          'mean_part' : 'Average number of parts',
                          'set_nbr' : 'Number of sets'})

Fig.update_layout(showlegend=False)

Fig.update_traces(marker=dict(size=20))

Fig.show()

In [ ]:
df_parts = pd.read_csv("/kaggle/input/lego-database/parts.csv")
df_parts.head()

In [ ]:
df_color = pd.read_csv("/kaggle/input/lego-database/colors.csv")
df_sets = pd.read_csv("/kaggle/input/lego-database/sets.csv")
df_themes = pd.read_csv("/kaggle/input/lego-database/themes.csv")
df_inventory_sets = pd.read_csv("/kaggle/input/lego-database/inventory_sets.csv")
df_inventories = pd.read_csv("/kaggle/input/lego-database/inventories.csv")
df_inventory_parts = pd.read_csv("/kaggle/input/lego-database/inventory_parts.csv")
df_parts = pd.read_csv("/kaggle/input/lego-database/parts.csv")
df_part_categories = pd.read_csv("/kaggle/input/lego-database/part_categories.csv")

In [ ]:
df_themes = pd.read_csv("/kaggle/input/lego-database/themes.csv")
df_themes.head()

In [ ]:
df_inventory_sets.head()

2. What colors are associated with which theme?